In [2]:
import pandas as pd
pd.set_option('float_format', '{:f}'.format)

In [3]:
def get_cdr(path):
    cdr = pd.read_csv(path, sep=';', header=None)
    cdr.rename(columns={0:'DATE', 1:'HOUR', 2:'DURATION', 3:'DDD_USER_FROM',
                    4:'USER_FROM', 5:'DDD_USER_TO', 6:'USER_TO', 7: 'ANTENNA'},inplace=True)
    cdr = cdr.drop(8,1)
    return cdr

In [4]:
path_sjdr = '/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr/cdr_regiao_imediata_sjdr.csv'
cdr_sjdr = get_cdr(path_sjdr)
cdr_sjdr

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14
0,2013-03-21,11:51:30,0.370000,32,5B5F2C071D12AF13219DF5EBE05132AF,32,9FB3B96B6D5E16C9DD564AA3E84F1954,50141,117,LOCAL,OI,OI,PRE PAGO,300000533
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210
2,2013-03-21,17:33:20,0.330000,61,85D5C50A6D882CA8E4BB00BCA3574417,61,0D8583F810B9720A8032BB939F12B3FF,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210
3,2013-03-21,11:58:26,1.270000,61,6C10A9E9F325CAA3CCB7F9A0D6983D2A,61,B0C50ED1DEC9E06E4C64E7419DDC4B09,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210
4,2013-03-21,16:06:59,0.170000,61,29122DCF4189448865B03F24C2DFEAE6,61,E0B097D86BF9C02A23011610EBAF47FD,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9293311,2013-04-19,14:51:18,1.100000,32,88407A1D70978E30C7EABBAEAB1AAC70,32,B6489911B08BC46BE4961AE216AEC568,23461,77,LOCAL,OI,TNL,POS PAGO,300000292
9293312,2013-04-19,12:32:44,0.670000,82,B97BACF4DACB5EC2C85D387FC67866BB,82,C438210ABEE57B97AEAB00958919CF3F,23441,117,LOCAL,OI,OI,PRE PAGO,300000210
9293313,2013-04-19,22:29:57,6.280000,82,06D5FF5901283A69AD23C295D3E24EFB,82,1029D7F81B2BCE9812F20CBE5A01599E,23451,117,LOCAL,OI,OI,PRE PAGO,300000533
9293314,2013-04-19,01:56:58,1.580000,82,7670BF3A1F8C74B01779DDA73554BC8D,82,94253ECA00D72C7FBF8C6706192CF3F8,23451,117,LOCAL,OI,OI,PRE PAGO,300000210


In [7]:
def get_period(cdr):
    date_min = cdr['DATE'].min()
    date_max = cdr['DATE'].max()
    return date_min + ' - ' + date_max

'2013-03-21 - 2013-04-19'

In [8]:
def cdr_filtered(cdr):
    cdr = cdr[cdr['DURATION'].between(0.07, 120)]
    return cdr

In [13]:
def users_data(cdr):
    cdr = cdr_filtered(cdr)
    user_from = len(cdr['USER_FROM'].unique())
    user_to = len(cdr['USER_TO'].unique())
    users = len(pd.unique(cdr[['USER_FROM', 'USER_TO']].values.ravel('k')))
    print('Usuários que realizaram ligações {} \nUsuários que receberam ligações {}\ntotal de usuários distintos {}'.format(user_from, user_to, users))

Usuários que realizaram ligações 439488 
Usuários que receberam ligações 1191551
total de usuários distintos 1296213


In [16]:
def cdr_filter_by_quantity_of_call(cdr):
    cdr_filter_by_quantity_of_call = cdr.groupby('USER_FROM').size().reset_index().rename(columns={0:'QUANTITY_OF_CALLS'})

    cdr_filter_by_quantity_of_call = cdr_filter_by_quantity_of_call.loc[(
                                (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] >= 3) 
                                & (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] <= 500))]
    return cdr_filter_by_quantity_of_call

In [19]:
def position_metrics(cdr):
    cdr = cdr_filter_by_quantity_of_call(cdr)    
    mean = cdr['QUANTITY_OF_CALLS'].mean()
    median = cdr['QUANTITY_OF_CALLS'].median()
    mode = cdr['QUANTITY_OF_CALLS'].mode()
    print('Média: {} \nMediana: {} \nModa: {}'.format(mean, median, mode))
position_metrics(cdr_sjdr)

Média: 33.17112711809413 
Mediana: 11.0 
Moda: 0    3
dtype: int64
